In [0]:
import os
import sys


import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
import papermill as pm
import scrapbook as sb

from recommenders.utils.notebook_utils import is_databricks
from recommenders.utils.spark_utils import start_or_get_spark
from recommenders.datasets.criteo import load_spark_df
from recommenders.datasets.spark_splitters import spark_random_split

# Setup MML Spark
#from recommenders.utils.spark_utils import MMLSPARK_REPO, MMLSPARK_PACKAGE
#print(MMLSPARK_REPO)
#print(MMLSPARK_PACKAGE)
#packages = [MMLSPARK_PACKAGE]
#repos = [MMLSPARK_REPO]
#spark = start_or_get_spark(packages=packages, repositories=repos)
#dbutils = None
#print("MMLSpark version: {} from {}".format(MMLSPARK_PACKAGE,MMLSPARK_REPO))
#from mmlspark.train import ComputeModelStatistics
#from mmlspark.lightgbm import LightGBMClassifier

# Synapse.ml is the new version of mml
import synapse.ml
from synapse.ml.train import ComputeModelStatistics
from synapse.ml.lightgbm import LightGBMClassifier

print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))

System version: 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]
PySpark version: 3.1.2


In [0]:
#!pip list

Package             Version
------------------- --------------------
ansiwrap            0.8.4
appdirs             1.4.4
argon2-cffi         20.1.0
async-generator     1.10
attrs               20.3.0
backcall            0.2.0
bleach              3.3.0
boto3               1.16.7
botocore            1.19.7
Bottleneck          1.3.2
category-encoders   1.3.0
certifi             2020.12.5
cffi                1.14.5
chardet             4.0.0
click               8.0.3
cornac              1.14.1
cycler              0.10.0
Cython              0.29.23
dbus-python         1.2.16
decorator           5.0.6
defusedxml          0.7.1
distlib             0.3.2
distro              1.4.0
distro-info         0.23ubuntu1
entrypoints         0.3
facets-overview     1.0.0
filelock            3.0.12
huggingface-hub     0.4.0
hypothesis          6.36.1
idna                2.10
ipykernel           5.3.4
ipython             7.22.0
ipython-genutils    0.2.0
ipywidgets          7.6.3
jedi                0.17.2
J

In [0]:
# Criteo data size, it can be "sample" or "full"
DATA_SIZE = "sample"

# LightGBM parameters
# More details on parameters: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
NUM_LEAVES = 32
NUM_ITERATIONS = 50
LEARNING_RATE = 0.1
FEATURE_FRACTION = 0.8
EARLY_STOPPING_ROUND = 10

# Model name
MODEL_NAME = 'lightgbm_criteo.mml'

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSessionLightGBM").getOrCreate()
#spark.version()

from pyspark.dbutils import DBUtils
dbutils = DBUtils(spark)

dbutils.widgets.help()

dbutils.widgets provides utilities for working with notebook widgets. You can create
different types of widgets and get their bound value.

For more info about a method, use dbutils.widgets.help("methodName") .
 combobox(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a combobox input widget with a given name, default value and choices dropdown(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a dropdown input widget a with given name, default value and choices get(name: String): String -> Retrieves current value of an input widget getArgument(name: String, optional: String): String -> (DEPRECATED) Equivalent to get multiselect(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a multiselect input widget with a given name, default value and choices remove(name: String): void -> Removes an input widget from the notebook removeAll: void -> Removes all widgets in the notebook text(name: String, defaultValue: String, label: String): void -> Creates a text input widget with a given name and default value

In [0]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark, dbutils=dbutils)
# visualize data
raw_data.limit(2).toPandas().head()

100%|██████████| 8.58k/8.58k [00:02<00:00, 2.95kKB/s]


,label,int00,int01,int02,int03,int04,int05,int06,int07,int08,...,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25
0,0,1,1,5,0,1382,4,15,2,181,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,None,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2,0,44,1,102,8,2,2,4,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,None,3a171ecb,43f13e8b,e8b83407,731c3655


In [0]:
raw_train, raw_test = spark_random_split(raw_data, ratio=0.8, seed=42)

In [0]:
columns = [c for c in raw_data.columns if c != 'label']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')

In [0]:
train = feature_processor.transform(raw_train)
test = feature_processor.transform(raw_test)

In [0]:
lgbm = LightGBMClassifier(
    labelCol="label",
    featuresCol="features",
    objective="binary",
    isUnbalance=True,
    boostingType="gbdt",
    boostFromAverage=True,
    baggingSeed=42,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION,
    earlyStoppingRound=EARLY_STOPPING_ROUND
)

In [0]:
model = lgbm.fit(train)

In [0]:
predictions = model.transform(test)

In [0]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("label")
    .setEvaluationMetric("AUC")
)

result = evaluator.transform(predictions)
auc = result.select("AUC").collect()[0][0]
result.show()

+---------------+------------------+
|evaluation_type|               AUC|
+---------------+------------------+
| Classification|0.6213613998849201|
+---------------+------------------+



In [0]:
# Record results with papermill for tests
sb.glue("auc", auc)

In [0]:
# save model
pipeline = PipelineModel(stages=[feature_processor, model])
pipeline.write().overwrite().save(MODEL_NAME)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-1623646647458095> in <module>
      1 # save model
      2 pipeline = PipelineModel(stages=[feature_processor, model])
----> 3 pipeline.write().overwrite().save(MODEL_NAME)

/databricks/spark/python/pyspark/ml/util.py in save(self, path)
    306         if not isinstance(path, str):
    307             raise TypeError("path should be a string, got type %s" % type(path))
--> 308         self._jwrite.save(path)
    309 
    310     def overwrite(self):

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    115     d